# Simple Model Example

There are two ways of running a simple eCalc™ model:

1. Specifying a YAML model
2. Specifying a Python model

Here we will look at both scenarios:

## The Simple Model
The Simple used in this example is the reference case in [eCalc™ Docs - Simple Model Example](https://equinor.github.io/ecalc/docs/about/modelling/examples/simple).

The model consists of a single installation with:
- Flare
- Gas Export Compressor
- Generator set A:
    - Base Production Load
    - Gas Injection Compressor
    - Produced Water Re-injection Pump
    - Seawater Injection Pump

The model.yaml file contains references to:
- production_data.csv
- genset.csv
- compressor_sampled.csv
- compressor_sampled_with_turbine.csv
- pump_sampled.csv
- pump_chart.csv

## Yaml Model
A YAML model consists of a main YAML file and may reference to other input files - usually CSV-files for timeseries inputs.
For more details about creating a model in YAML, see the [eCalc™ Docs](https://equinor.github.io/ecalc/docs/about)

All files for the example can be found in the directory references in the code below.

Here is how you load and run a YAML-model in eCalc™

In [ ]:
from pathlib import Path
from libecalc.core.graph_result import GraphResult
from libecalc.core.ecalc import EnergyCalculator
from libecalc.common.time_utils import Frequency
from libecalc.presentation.yaml.model import YamlModel
from libecalc.examples import simple


model_path = Path(simple.__file__).parent / "model.yaml"
yaml_model = YamlModel(path=model_path, output_frequency=Frequency.NONE)

model = EnergyCalculator(graph=yaml_model.graph)
consumer_results = model.evaluate_energy_usage(yaml_model.variables)
emission_results = model.evaluate_emissions(
    variables_map=yaml_model.variables,
    consumer_results=consumer_results,
)
result = GraphResult(
    graph=yaml_model.graph,
    consumer_results=consumer_results,
    variables_map=yaml_model.variables,
    emission_results=emission_results,
)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

print("Iterating the model components to plot results: \n")
for identity, component in yaml_model.graph.nodes.items():
    if identity in result.consumer_results:
        component_result = result.consumer_results[identity].component_result
        ds = pd.Series(component_result.energy_usage.values, index=component_result.energy_usage.timesteps)
        _ = ds.plot(xlabel="time", ylabel=component_result.energy_usage.unit, title=f"Component: {component.name} type: {type(component).__name__}", kind="line")
        plt.show(block=False)  # block=False in order to run in CI-tests.

In [ ]:
# Dummy test in order to test to assert that this notebook runs in GitHub Actions
def test_notebook_works():
    assert True